In [1]:
import sys
sys.path.append("/Users/tamara/Documents/Projects/movie-chatbot")

In [2]:
import pandas as pd
from pathlib import Path

# Movies Titles

In [3]:
import polars as pl

path = Path("/Users/tamara/Documents/Projects/movie-chatbot/data/raw")

movies = pl.read_csv(path / "movies_context_*.csv").drop("")

In [4]:
movies.columns

['Title',
 'Release date',
 'Genre',
 'Runtime',
 'Language',
 'Link',
 'parse_ts',
 'TitleId',
 'SourceYear']

In [5]:
movies.shape

(610, 9)

In [6]:
def fix_shifted_data(movies: pl.DataFrame) -> pl.DataFrame:
    is_shifted = pl.col("Release date").str.replace(r'(\[\d+\])+', '').str.strip_chars().str.strptime(dtype=pl.Datetime, format='%B %d, %Y', strict=False).is_null()
    shifted_data = movies.filter(is_shifted).to_pandas()
    not_shifted_data = movies.filter(~is_shifted).to_pandas()

    shifted_data.iloc[:, 2:-3] = shifted_data.iloc[:, 1:-4]
    shifted_data.iloc[:, 1] = None

    return pl.from_pandas(pd.concat([
        shifted_data,
        not_shifted_data
    ]))


In [7]:
runtime_pattern = "(?:(\d+)\s?h)?(?:\s?(\d+)\s?min)?"

fix_shifted_data(
    movies.filter(pl.col("Title") != "Awaiting release")
).with_columns(
    // pl.col("Title").alias("title"),
    // pl.col("Title").alias("title"),
    
    pl.col("Release date").str.replace(r'(\[\d+\])+', '').str.strip_chars().str.strptime(dtype=pl.Datetime, format='%B %d, %Y', strict=False).alias("release_date"),
    pl.col("parse_ts").str.to_datetime().alias("parse_ts"),
    pl.col("SourceYear").str.replace("since_", "").cast(pl.Int32).alias("year"),
    pl.col('Title').str.replace(r'(\[\d+\])+', '').str.strip_chars(),
    (pl.col("Runtime").str.extract_groups(runtime_pattern).struct["1"].cast(pl.Int32) * 60 +
    pl.col("Runtime").str.extract_groups(runtime_pattern).struct["2"].cast(pl.Int32).fill_null(0)).alias("runtime_min"),
).sort(["year", "TitleId"]).with_columns(
    pl.col("release_date").forward_fill()
).with_columns(
    (pl.col('Title') + pl.lit(" is a ") + pl.col('Genre') + pl.lit(" Netflix movie in ") + pl.col('Language') +
      + pl.lit(" language that was released on ") + pl.col('release_date').dt.strftime("%Y-%m-%d")).alias('text'),
) 
# .select(["Title", "Genre", "Language", "SourceYear", "TitleId", "year", "release_date", "runtime_min", "text", "parse_ts"])

SyntaxError: invalid syntax (220988126.py, line 6)

In [10]:
movies_clean

NameError: name 'movies_clean' is not defined

In [294]:
movies_clean.null_count()

Title,Release date,Genre,Runtime,Language,Link,parse_ts,TitleId,SourceYear,release_date,year,text
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,92,0,0,0,155,0,0,0,0,0,0


In [295]:
movies_clean.filter(pl.col("Title") == "The Platform 2")

Title,Release date,Genre,Runtime,Language,Link,parse_ts,TitleId,SourceYear,release_date,year,text
str,str,str,str,str,str,datetime[μs],i64,str,datetime[μs],i32,str
"""The Platform 2""",null,"""Science fiction""","""1 h 40 min""","""Spanish""","""/wiki/The_Platform_2""",2024-10-14 00:00:00,99,"""since_2024""",2024-10-04 00:00:00,2024,"""The Platform 2 is a Science fi…"


In [244]:
movies_clean.filter(pl.col("rutime_min").is_null())["Runtime"].unique()

Runtime
str
"""TBA"""


# Movies Data

In [245]:
import json 

with open(path / "movies_data_2021.json", "r") as f:
    movies_data = json.load(f)

In [254]:
import json 

movies_data_l = []
meta_cols = ["TitleId", "SourceYear", "Title"]

years = ["2021", "2022", "2023", "since_2024"]
for year in years:
    with open(path / f"movies_data_{year}.json", "r") as f:
        movies_data = json.load(f)

    for movie_info in movies_data:
        for topic, topic_text in movie_info.items():
            if topic not in meta_cols:
                movies_data_l.append({
                    "TitleId": movie_info["TitleId"],
                    "SourceYear": movie_info["SourceYear"],
                    "topic": topic,
                    "text": topic_text
                })


movies_clean_cols = ["TitleId", "SourceYear", "Title", "Genre", "runtime_min", "Language", "release_date"]
movies_data_df = pl.DataFrame(movies_data_l).join(movies_clean.select(movies_clean_cols), on=["TitleId", "SourceYear"], how="left")
            

In [255]:
movies_data_df

TitleId,SourceYear,topic,text,Title,Genre,runtime_min,Language,release_date
i64,str,str,str,str,str,i32,str,datetime[μs]
1,"""2021""","""General""","""Pieces of a Woman is a 2020 dr…","""Pieces of a Woman""","""Drama""",126,"""English""",2021-01-07 00:00:00
1,"""2021""","""Plot""","""Martha and Sean, a young Bosto…","""Pieces of a Woman""","""Drama""",126,"""English""",2021-01-07 00:00:00
1,"""2021""","""Cast""","""Vanessa Kirby as Martha Weiss …","""Pieces of a Woman""","""Drama""",126,"""English""",2021-01-07 00:00:00
1,"""2021""","""Production-Play""","""The play Pieces of a Woman was…","""Pieces of a Woman""","""Drama""",126,"""English""",2021-01-07 00:00:00
1,"""2021""","""Production-Development and the…","""The film Pieces of a Woman was…","""Pieces of a Woman""","""Drama""",126,"""English""",2021-01-07 00:00:00
…,…,…,…,…,…,…,…,…
127,"""since_2024""","""Production""","""In June 2022, it was reported …","""Back in Action""","""Action comedy""",null,"""English""",2025-01-17 00:00:00
127,"""since_2024""","""Release""","""Back in Action is scheduled to…","""Back in Action""","""Action comedy""",null,"""English""",2025-01-17 00:00:00
128,"""since_2024""","""General""","""The Witcher: Sirens of the Dee…","""The Witcher: Sirens of the Dee…","""Animation""",null,"""English""",2025-02-11 00:00:00


In [270]:
movies_data_df.select(
    pl.col("text").str.len_chars().alias("text_chars"),
    pl.col("text").str.split(" ").list.len().alias("text_words"),
    pl.col("text").str.len_bytes().alias("text_bytes")
).describe()

statistic,text_chars,text_words,text_bytes
str,f64,f64,f64
"""count""",2081.0,2081.0,2081.0
"""null_count""",0.0,0.0,0.0
"""mean""",1010.975012,177.966362,1014.003364
"""std""",1295.960976,225.523538,1298.179201
"""min""",11.0,4.0,11.0
"""25%""",263.0,48.0,265.0
"""50%""",499.0,89.0,501.0
"""75%""",1175.0,204.0,1177.0
"""max""",15584.0,2687.0,15653.0
